In [9]:
import os, sys
import gc
import time

import torch
gc.collect()
# os.environ['CUDA_LAUNCH_BLOCKING'] = "4"
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"  
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
torch.cuda.set_device(device)
# print(device)

from sklearn.metrics import accuracy_score
import swifter

from tqdm.notebook import tqdm
tqdm.pandas()
from nltk.tokenize import word_tokenize


from utils.utils_init_dataset import set_seed, load_dataset_loader
from utils.utils_semantic_use import USE
from utils.utils_data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader, EmotionDetectionDataset, EmotionDetectionDataLoader
from utils.utils_metrics import document_sentiment_metrics_fn
from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, logit_prob, load_word_index
from utils.get_args import get_args

# from attack.adv_attack import attack
import os, sys

from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, logit_prob, load_word_index
import time
from nltk.tokenize import word_tokenize
import torch
import torch.nn.functional as F
import numpy as np
import math

from attack.attack_helper import get_synonyms, codemix_perturbation, codemix_perturbation_cache, synonym_replacement, swap_minimum_importance_words, trans_dict, read_dict

# from attack.adv_attack import attack
import main

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words_set = []
for w in stopwords.words('indonesian'):
    stop_words_set.append(w)

from icecream import ic
import pandas as pd
pd.set_option('display.max_colwidth', None)
from itertools import chain
from deep_translator import GoogleTranslator


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/m13518040/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
def attack(text_ls,
           true_label,
           predictor,
           tokenizer,
           att_ratio,
           perturbation_technique,
           translator=None,
           lang_codemix=None,
           sim_predictor=None,
           sim_score_threshold=0,
           sim_score_window=15,
           batch_size=32, 
           import_score_threshold=-1.):
    
    start_time = time.time()
    label_dict = {
        'positive': 0, 
        'neutral': 1, 
        'negative': 2}
    
    original_text = text_ls
    splitted_text = text_ls.split()
    
    orig_label, orig_probs, orig_prob = logit_prob(text_ls, predictor, tokenizer)
    
    if len(splitted_text) > sim_score_window:
        sim_score_threshold = 0.1  
    #     SEK SALAAHHHHHH
    if true_label != orig_label:
        running_time = round(time.time() - start_time, 2)
        # perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time
        orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
        return original_text, 1.000, orig_label, orig_probs, orig_label, orig_probs,'', running_time
    else:
        text_ls = splitted_text
#         isalnum karena kata yg ada nonalnum nya punya prediksi yang ga konsisten
        text_ls = [word for word in text_ls if word.isalnum()]
        len_text = len(text_ls)
        
        half_sim_score_window = (sim_score_window - 1) // 2
        
        leave_1_texts = [' '.join(text_ls[:ii] + [tokenizer.mask_token] + text_ls[min(ii + 1, len_text):]) for ii in range(len_text)]
        
        ic(leave_1_texts)
        
        leave_1_probs = []
        leave_1_probs_argmax = []
        
        if len(text_ls) <=1:
            leave_1_probs_argmax, leave_1_probs, _ = logit_prob(leave_1_texts, predictor, tokenizer)
            leave_1_probs_argmax = [leave_1_probs_argmax]
            leave_1_probs = [leave_1_probs.detach().cpu().numpy()]
            # ic(leave_1_probs)
            leave_1_probs = torch.tensor(np.array(leave_1_probs)).to("cuda:0")
        else:
            leave_1_probs_argmax, leave_1_probs, _ = logit_prob(leave_1_texts, predictor, tokenizer,batch=True)
            leave_1_probs = torch.tensor(np.array(leave_1_probs.detach().cpu().numpy())).to("cuda:0")
        
        orig_prob_extended=np.empty(len_text)
        orig_prob_extended.fill(orig_prob)
        
        orig_prob_extended = torch.tensor(orig_prob_extended).to("cuda:0")
        
        arr1 = orig_prob_extended - leave_1_probs[:,orig_label] + float(leave_1_probs_argmax != orig_label)
        arr2 = (leave_1_probs.max(dim=-1)[0].to("cuda:0") - orig_probs[leave_1_probs_argmax].to("cuda:0"))
        
        import_scores = arr1*arr2
        import_scores = [im * -1 for im in import_scores]
        
        words_perturb = []
        for idx, score in sorted(enumerate(import_scores), key=lambda x: x[1], reverse=True):
            try:
                if score > import_score_threshold and text_ls[idx] not in stop_words_set:
                    words_perturb.append((idx, text_ls[idx], score.item()))
            except Exception as e:
                print(e)
                print(idx, len(text_ls), import_scores.shape, text_ls, len(leave_1_texts))
        
        # ic(sorted(words_perturb, key=lambda x: x[2]))
        ic(words_perturb)
        
        num_perturbation = math.floor(len(words_perturb)*att_ratio)
        
        if num_perturbation < 1:
            num_perturbation = 1
        
#       top words perturb berisi list kata terpenting yang tidak akan diswitch ketika first_codemix_sim_score < sim_score_threshold
        top_words_perturb = words_perturb[:num_perturbation]
    
        ic(top_words_perturb)
        
        trans_word = [twp[1] for twp in top_words_perturb]
        
        if perturbation_technique == "codemixing":
            perturbed_text,translated_words = codemix_perturbation_cache(text_ls, lang_codemix, top_words_perturb, translator)
        elif perturbation_technique == "synonym_replacement":
            perturbed_text,translated_words = synonym_replacement(text_ls, top_words_perturb)
        
        ic(perturbed_text)
        
        first_perturbation_sim_score = sim_predictor.semantic_sim(original_text, perturbed_text)
        
        ic(first_perturbation_sim_score)
        
        words_perturb_candidates = []
        if first_perturbation_sim_score < sim_score_threshold:
            words_perturb_candidates.append(top_words_perturb)
            swapped = swap_minimum_importance_words(words_perturb, top_words_perturb)
            for s in swapped:
                words_perturb_candidates.append(s)

            words_perturb_candidates = [[tuple(w) for w in wpc] for wpc in words_perturb_candidates]

            candidate_comparison = {}
            for wpc in words_perturb_candidates:
                
                if perturbation_technique == "codemixing":
#                     bisa dicache, gaperlu request satu2
                    perturbed_candidate,translated_words = codemix_perturbation_cache(text_ls, lang_codemix, wpc, translator)
                elif perturbation_technique == "synonym_replacement":
                    perturbed_candidate,translated_words = synonym_replacement(text_ls, wpc)
                    
                perturbed_candidate_sim_score = sim_predictor.semantic_sim(original_text, perturbed_candidate)
                
                candidate_comparison[perturbed_candidate] = (perturbed_candidate_sim_score, wpc[-1][-1], wpc)

            sorted_candidate_comparison = sorted(candidate_comparison.keys(), key=lambda x: (float(candidate_comparison[x][0]), float(candidate_comparison[x][1])), reverse=True)

            perturbed_text = sorted_candidate_comparison[0]
            
            top_words_perturb = candidate_comparison[sorted_candidate_comparison[0]][-1]
            trans_word = [twp[1] for twp in top_words_perturb]
            
        perturbed_semantic_sim = sim_predictor.semantic_sim(original_text, perturbed_text)
        if perturbed_semantic_sim < sim_score_threshold:
            perturbed_text = original_text
            perturbed_semantic_sim = 1.000
        
        perturbed_label, perturbed_probs, perturbed_prob = logit_prob(perturbed_text, predictor, tokenizer)
        
        orig_probs = np.round(orig_probs.detach().cpu().numpy().tolist(),4)
        perturbed_probs = np.round(perturbed_probs.detach().cpu().numpy().tolist(),4)
        
        running_time = round(time.time() - start_time, 4)
        
        return perturbed_text, perturbed_semantic_sim, orig_label, orig_probs, perturbed_label, perturbed_probs, translated_words, running_time

In [29]:
attack(
    text_ls = row_text,
    true_label = row_sentiment,
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = perturb_ratio,
    perturbation_technique = perturbation_technique,
    translator=translator,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

ic| leave_1_texts: ['[MASK] sini saya suka makan nasi bakmoy nya kroket nya juga enak kok kisaran '
                    'harga kroket nya rp rp 4500',
                    'di [MASK] saya suka makan nasi bakmoy nya kroket nya juga enak kok kisaran '
                    'harga kroket nya rp rp 4500',
                    'di sini [MASK] suka makan nasi bakmoy nya kroket nya juga enak kok kisaran '
                    'harga kroket nya rp rp 4500',
                    'di sini saya [MASK] makan nasi bakmoy nya kroket nya juga enak kok kisaran '
                    'harga kroket nya rp rp 4500',
                    'di sini saya suka [MASK] nasi bakmoy nya kroket nya juga enak kok kisaran '
                    'harga kroket nya rp rp 4500',
                    'di sini saya suka makan [MASK] bakmoy nya kroket nya juga enak kok kisaran '
                    'harga kroket nya rp rp 4500',
                    'di sini saya suka makan nasi [MASK] nya kroket nya juga enak kok kisaran '
         

('di sini saya suka makan nasi bakmoy nya kroket nya juga enak kok kisaran harga kroket nya rp rp 4500',
 0.98318493,
 0,
 array([9.976e-01, 2.000e-03, 4.000e-04]),
 0,
 array([9.976e-01, 1.900e-03, 5.000e-04]),
 {'enak': 'enak', 'suka': 'suka', 'harga': 'harga', 'nya': 'nya'},
 0.6202)

In [4]:
model_target="IndoBERT"
downstream_task="sentiment"
attack_strategy="adversarial"
finetune_epoch=1
num_sample=50
# exp_name=
perturbation_technique="codemixing"
perturb_ratio=0.4
perturb_lang="en"
seed=26092020

In [11]:
set_seed(seed)
use = USE()

tokenizer, config, finetuned_model = init_model(model_target, downstream_task, seed)
w2i, i2w = load_word_index(downstream_task)

train_dataset, train_loader, train_path = load_dataset_loader(downstream_task, 'train', tokenizer)
valid_dataset, valid_loader, valid_path = load_dataset_loader(downstream_task, 'valid', tokenizer)
test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)
# print(model.cuda())
# finetuned_model = fine_tuning_model(model, i2w, train_loader, valid_loader, finetune_epoch)

# exp_dataset.to_csv(os.getcwd() + r'/result/'+exp_name+".csv", index=False)

In [12]:
exp_dataset = valid_dataset.load_dataset(valid_path).head(num_sample)
# exp_dataset

In [14]:
text,label = None,None

row_text = "di sini saya suka makan nasi bakmoy nya . kroket nya juga enak kok . kisaran harga kroket nya rp 3.000 - rp 4500"
row_sentiment = 0

translator = None
if perturbation_technique == "codemixing":
    translator = read_dict(os.getcwd() + r"/dicts/new_dict_"+perturb_lang+".txt")
# print(perturbation_technique)
if downstream_task == 'sentiment':
    text = 'text'
    label = 'sentiment'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row_text,
            true_label = row_sentiment,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            translator=translator,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )
elif downstream_task == 'emotion':
    text = 'tweet'
    label = 'label'
    exp_dataset[['perturbed_text', 'perturbed_semantic_sim', 'pred_label', 'pred_proba', 'perturbed_label', 'perturbed_proba', 'translated_word(s)', 'running_time(s)']] = exp_dataset.progress_apply(
        lambda row: attack(
            text_ls = row.tweet,
            true_label = row.label,
            predictor = finetuned_model,
            tokenizer = tokenizer, 
            att_ratio = perturb_ratio,
            perturbation_technique = perturbation_technique,
            translator=translator,
            lang_codemix = perturb_lang,
            sim_predictor = use), axis=1, result_type='expand'
    )

before_attack = accuracy_score(exp_dataset[label], exp_dataset['pred_label'])
after_attack = accuracy_score(exp_dataset[label], exp_dataset['perturbed_label'])

exp_dataset.loc[exp_dataset.index[0], 'before_attack_acc'] = before_attack
exp_dataset.loc[exp_dataset.index[0], 'after_attack_acc'] = after_attack
exp_dataset.loc[exp_dataset.index[0], 'avg_running_time(s)'] = exp_dataset["running_time(s)"].mean()

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
main(
    model_target="IndoBERT",
    downstream_task="sentiment",
    attack_strategy="adversarial",
    finetune_epoch=1,
    num_sample=2,
    exp_name="fixx",
    perturbation_technique="codemixing",
    perturb_ratio=0.4,
    perturb_lang="en",
)

In [ ]:
txt = "betapa bejad kader gerindra yang anggota dprd mencabuli anak smp , rakyat harus cerdas partai mana yang harus di tengelamkan di 2019"
true_label = 2

perturbed_text, perturbed_semantic_sim, pred_label, pred_proba, perturbed_label, perturbed_proba, translated_words, running_time = attack(
    text_ls = txt,
    true_label = true_label,
    predictor = finetuned_model,
    tokenizer = tokenizer, 
    att_ratio = 0.8,
    perturbation_technique = perturbation_technique,
    lang_codemix = perturb_lang,
    sim_predictor = use
)

ic(perturbed_text)
ic(perturbed_semantic_sim)
ic(pred_label)
ic(pred_proba)
ic(perturbed_label)
ic(perturbed_proba)
ic(translated_words)
ic(running_time)

In [ ]:
exp_dataset

In [ ]:
exp_dataset.to_csv(os.getcwd() + r'/result/'+"test"+".csv", index=False)